# W2V: Song Recommend System

In [1]:
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent.parent)) 

import pandas as pd
from urllib import request

from src.fs import read_pickle, write_pickle

## Read: Playlist

In [3]:
# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')
data

In [4]:
# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]
len(lines)

11138

In [5]:
lines[:5]

['0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 2 42 43 44 45 46 47 48 20 49 8 50 51 52 53 54 55 56 57 25 58 59 60 61 62 3 63 64 65 66 46 47 67 2 48 68 69 70 57 50 71 72 53 73 25 74 59 20 46 75 76 77 59 20 43 ',
 '78 79 80 3 62 81 14 82 48 83 84 17 85 86 87 88 74 89 90 91 4 73 62 92 17 53 59 93 94 51 50 27 95 48 96 97 98 99 100 57 101 102 25 103 3 104 105 106 107 47 108 109 110 111 112 113 25 63 62 114 115 84 116 117 118 119 120 121 122 123 50 70 71 124 17 85 14 82 48 125 47 46 72 53 25 73 4 126 59 74 20 43 127 128 129 13 82 48 130 131 132 133 134 135 136 137 59 46 138 43 20 139 140 73 57 70 141 3 1 74 142 143 144 145 48 13 25 146 50 147 126 59 20 148 149 150 151 152 56 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 60 176 51 177 178 179 180 181 182 183 184 185 57 186 187 188 189 190 191 46 192 193 194 195 196 197 198 25 199 200 49 201 100 202 203 204 205 206 207 32 208 20

In [5]:
# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]
len(playlists)

11088

In [6]:
playlists[0][:5]

['0', '1', '2', '3', '4']

In [ ]:
write_pickle(playlists, "data/playlists.pkl")

In [4]:
playlists = read_pickle("data/playlists.pkl")

## Read: Songs

In [8]:
# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')

In [9]:
songs_file[:5]

['0 \tGucci Time (w\\/ Swizz Beatz)\tGucci Mane',
 '1 \tAston Martin Music (w\\/ Drake & Chrisette Michelle)\tRick Ross',
 '2 \tGet Back Up (w\\/ Chris Brown)\tT.I.',
 '3 \tHot Toddy (w\\/ Jay-Z & Ester Dean)\tUsher',
 '4 \tWhip My Hair\tWillow']

In [10]:
songs = [s.rstrip().split('\t') for s in songs_file]
songs[:3]

[['0 ', 'Gucci Time (w\\/ Swizz Beatz)', 'Gucci Mane'],
 ['1 ', 'Aston Martin Music (w\\/ Drake & Chrisette Michelle)', 'Rick Ross'],
 ['2 ', 'Get Back Up (w\\/ Chris Brown)', 'T.I.']]

In [11]:
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')
songs_df.head()

,title,artist
id,,
0,Gucci Time (w\/ Swizz Beatz),Gucci Mane
1,Aston Martin Music (w\/ Drake & Chrisette Mich...,Rick Ross
2,Get Back Up (w\/ Chris Brown),T.I.
3,Hot Toddy (w\/ Jay-Z & Ester Dean),Usher
4,Whip My Hair,Willow


In [12]:
songs_df.to_pickle("data/songs_df.pkl")

In [3]:
songs_df = pd.read_pickle("data/songs_df.pkl")

## Word2Vec Model

In [5]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [7]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [6]:
song_id = 2172

# Ask the model for songs similar to song #2172
model.wv.most_similar(positive=str(song_id))

[('2849', 0.9985162615776062),
 ('3167', 0.9975162744522095),
 ('5586', 0.9967291355133057),
 ('6624', 0.9962368607521057),
 ('2976', 0.9961584210395813),
 ('11473', 0.995529294013977),
 ('5634', 0.995094358921051),
 ('2640', 0.9945476651191711),
 ('6658', 0.9942667484283447),
 ('2704', 0.9940786957740784)]

In [8]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
3167,Unchained,Van Halen
5586,The Last In Line,Dio
6624,Everybody Wants Some!!!,Van Halen
2976,I Don't Know,Ozzy Osbourne
